In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name='real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)

retriever = vector_store.as_retriever()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model='gpt-4o',
                 temperature=0
)


template = """
당신은 질문-응답 작업을 수행하는 어시스턴트입니다.
아래에 제공된 검색된 컨텍스트를 사용하여 질문에 답변하세요.
답을 모를 경우, 모른다고 말하세요.
최대 두 문장으로, 간결하게 답변하세요.

질문: {question}
컨텍스트: {context}
답변:
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [28]:
from datasets import Dataset

questions = [
    "종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?", 
    "1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은 무엇인가?",
    "주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?",
]

gt = [
    "종합부동산세는 고액 부동산 보유자에 대한 조세 형평성과 부동산 가격 안정을 목적으로 한다.",
    "1세대 1주택자는 공시가격 요건과 연령 및 보유기간 요건을 충족하면 세액 공제를 받을 수 있다.",
    "3주택 이상 보유한 개인에게는 누진적인 고율의 종합부동산세 세율이 적용된다."
]
answers = []
contexts = []

for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append(
        [doc.page_content for doc in retriever.get_relevant_documents(query)]
    )

data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    'ground_truth': gt
}

dataset = Dataset.from_dict(data)


In [31]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
     llm=llm  
)

df = result.to_pandas()

/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/hq/rb1q4rn512qbbsyy87bd4g9c0000gn/T/ipykernel_4860/2919819784.py:2: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var

In [32]:
df

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,"종합부동산세의 부과 목적은 무엇이며, 재산세와 어떤 점에서 다른가?","[종합부동산세법\n[시행 2023. 4. 18] [법률 제19342호, 2023. ...",종합부동산세의 부과 목적은 고액의 부동산 보유자에게 세금을 부과하여 조세부담의 형평...,종합부동산세는 고액 부동산 보유자에 대한 조세 형평성과 부동산 가격 안정을 목적으로...,0.6,0.848478,1.000000,1.0
1,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위한 요건은...,[과세기준일 현재 만 60세 이상인 1세대 1주택자가 제8조제4항 각 호의 어느 하...,1세대 1주택자가 종합부동산세 과세대상에서 제외되거나 세액 공제를 받기 위해서는 과...,1세대 1주택자는 공시가격 요건과 연령 및 보유기간 요건을 충족하면 세액 공제를 받...,0.2,0.849387,1.000000,1.0
2,주택을 3채 이상 보유한 개인에게 적용되는 종합부동산세 세율 구조는 어떻게 되는가?,[납세의무자가 2주택 이하를 소유한 경우\n\n| 과세표준 ...,주택을 3채 이상 보유한 개인의 종합부동산세 세율 구조는 다음과 같습니다: 과세표준...,3주택 이상 보유한 개인에게는 누진적인 고율의 종합부동산세 세율이 적용된다.,1.0,0.861491,0.833333,1.0


In [ ]:
#!pip install datasets
#!pip install ragas


  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-1.1.4-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which 